In [8]:
from sklearn.model_selection import train_test_split
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, '../src')

from pathlib import Path
import pandas as pd
from zipfile import ZipFile
import numpy as np
import matplotlib.pyplot as plt
import copy


PNS_DATA = Path('../data/processed/PNS_2019/all.zip').resolve()
zip_pns = ZipFile(PNS_DATA)
df = pd.read_csv(zip_pns.open(zip_pns.namelist()[0]), sep=",", low_memory=False)

PATH_GEN_DATA = Path('generated_data/pns_data_models').resolve()

def process_df(df_i, remove_na=True):
    df_i = copy.deepcopy(df_i)
    if remove_na:
        df_i = df_i.dropna()
    return df_i.drop(['r_norte', 'r_nordeste', 'r_sudeste', 'r_sul', 'r_centro_oeste'], axis=1)



# Target
df['depression'] = np.where(df['phq9_total'].isna(), np.nan, np.where(df['phq9_total'] >= 10, 1, 0))
df = df.dropna(subset=['depression'])
df['depression'].value_counts()



# Alimentos in natura ou minimamente processado
innatura = ['P00601','P00602', 'P00603', 'P00604', 'P00605', 'P00607', 'P00608', 'P00609', 'P00610', 'P00611', 'P00612', 'P00613']
df_innatura = df[innatura]
df_innatura['qst_innatura'] = (df_innatura == 1.0).sum(axis=1)
# Alimentos ultraprocessados
ultraprocessados = ['P00614', 'P00615', 'P00616', 'P00617', 'P00618', 'P00619', 'P00620', 'P00621', 'P00622', 'P00623']
df_upf = df[ultraprocessados]
df_upf['qst_upf'] = (df_upf == 1.0).sum(axis=1)

# A literatura não usa a porcentagem de ultraprocessados do prato como análise. O artigo 'Food consumption and depression among Brazilian adults: results from the Brazilian National Health Survey, 2013'
# Usa como base o número de perguntas respondidas 'Sim' nos alimentos ultraprocessados das ultimas 24hrs
df['per_innatura'] = df_innatura['qst_innatura'] / (df_innatura['qst_innatura'] + df_upf['qst_upf']) * 100
df['per_upf'] = df_upf['qst_upf'] / (df_innatura['qst_innatura'] + df_upf['qst_upf']) * 100
df['qst_upf'] = df_upf['qst_upf'] # Baseado no artigo

def classify_upf(row):
    value = row['qst_upf']

    if  0 <= value < 2:
        return 1
    elif value < 4:
        return 2
    elif value < 6:
        return 3
    elif value < 8:
        return 4
    elif value <= 10:
        return 5

    return np.nan # questionario não respondido

df['upf_severity'] = df.apply(classify_upf, axis=1)


def count_imc(row):
    height = row['altura']
    weight = row['peso']

    imc = weight/np.square(height/100)   # Peso está em kg e altura em cm, no calculo do imc é kg/m^2
    return imc

df['imc'] = df.apply(count_imc, axis=1)


def classify_imc(row):
    value = row['imc']

    if  0 <= value < 18.5:
        return 1  # Baixo peso
    elif value < 24.9:
        return 2 # Eutrofia/peso adequado
    elif value < 29.9:
        return 3   # Sobrepeso
    elif value >= 30:
        return 4  # Obesidade

    return np.nan # questionario não respondido

df['class_imc'] = df.apply(classify_imc, axis=1)


def classify_idade(row):
    value = row['idade']
    if  18 <= value <= 29:
        return 1
    elif value <= 39:
        return 2
    elif value <= 49:
        return 3
    elif value <= 59:
        return 4

    return np.nan

df['class_idade'] = df.apply(classify_idade, axis=1)


# Essa variável foi criada a partir de um índice utilizado pela OMS. Na literatura, usam a pergunta 'P034' relacionada à prática de exercícios físicos nos ultimos 3 meses
df_activity = pd.DataFrame()
df_activity['minutes_per_week'] = df['P03702'] * df['P035']

vigorous_activities = [3.0, 4.0, 5.0, 6.0, 12.0, 13.0, 15.0]
moderated_activities = [1.0, 2.0, 7.0, 8.0, 9.0, 10.0, 11.0, 14.0, 16.0, 17.0]
df_activity['time_vigorous'] = df_activity['minutes_per_week'].loc[df['P036'].isin(vigorous_activities)]
df_activity['time_moderated'] = df_activity['minutes_per_week'].loc[df['P036'].isin(moderated_activities)]

df_activity['activity_class'] = 0
df_activity.loc[df_activity['minutes_per_week'].isna(), 'activity_class'] = np.nan
df_activity.loc[(df_activity['time_vigorous'] >= 75) | (df_activity['time_moderated'] >= 150), 'activity_class'] = 1

df['activity_class'] = df_activity['activity_class']
df['exerc_fisico'] = np.where(df['P034'].isin([1.0]), 1, np.where(df['P034'] == 2.0, 0, np.nan))
df['tabagismo'] = np.where(df['P050'].isin([1.0, 2.0]), 1, np.where(df['P050'] == 3.0, 0, np.nan))

# Na literatura eles usam outra variável para analisar o consumo alcoolico
# df['cons_alcool'] = np.where(df['P02801'] >= 1, 1, 0)
# df['cons_alcool'] = np.where(df['P02801'].isna(), np.nan, df['cons_alcool'])

# Olhar artigo 'Food consumption and depression among Brazilian adults: results from the Brazilian National Health Survey, 2013'
df['cons_alcool'] = np.where(df['P027'].isin([2.0, 3.0]), 1, np.where(df['P027'] == 1.0, 0, np.nan))

df = df[df['P005'] != 1.0] #Retirando as grávidas da análise

df['obesidade'] = np.where(df['class_imc'].isna(), np.nan, np.where(df['class_imc'] >= 4, 1, 0))

# Limitação da análise por idades que se adequam ao PHQ9 baseado no artigo 'Food consumption and depression among Brazilian adults: results from the Brazilian National Health Survey, 2013'
df = df[(df['idade'] >= 18) & (df['idade'] <= 59)]


regiao_mapping = {
    'r_norte': [11, 12, 13, 14, 15, 16, 17],
    'r_nordeste': [21, 22, 23, 24, 25, 26, 27, 28, 29],
    'r_sudeste': [31, 32, 33, 35],
    'r_sul': [41, 42, 43],
    'r_centro_oeste': [50, 51, 52, 53]
}

for regiao, estados in regiao_mapping.items():
    df[regiao] = df['uf'].apply(lambda x: int(x in estados))


df['class_renda'] = np.where(df['renda_pc'].isin([4.0, 5.0, 6.0, 7.0]), 1, np.where(df['renda_pc'].isin([1.0, 2.0, 3.0]), 0, np.nan))





df_total = pd.DataFrame()

df_total['exerc_fisico'] = df['exerc_fisico']
df_total['tabagismo'] = df['tabagismo']
df_total['cons_alcool'] = df['cons_alcool']
df_total['cancer'] = df['Q120']
df_total['hipertensao'] = df['Q00201']
df_total['diabetes'] = df['Q03001']
df_total['cardiovascular'] = df['Q06306']
df_total['hipercolesterolemia'] = df['Q060']
df_total['avc'] = df['Q068']
df_total['artrite'] = df['Q079']
df_total['obesidade'] = df['obesidade']
df_total['depression'] = df['depression']
df_total['sexo'] = df['sexo']
df_total['estado_civil'] = df['vive_conjugue']
df_total = df_total.replace(2.0, 0.0)
df_total['class_renda'] = df['class_renda']

df_total['r_norte'] = df['r_norte']
df_total['r_nordeste'] = df['r_nordeste']
df_total['r_sudeste'] = df['r_sudeste']
df_total['r_sul'] = df['r_sul']
df_total['r_centro_oeste'] = df['r_centro_oeste']

df_total['escolaridade'] = df['escolaridade_agregada']
df_total['upf'] = df['qst_upf']
df_total['class_idade'] = df['class_idade']

df_total['perc_saude'] = df['N00101'] # Variável não utilizada na literatura, meio duvidosa

df_train_total, df_test_total = train_test_split(df_total, test_size=0.3, random_state=42)

df_total_t = copy.deepcopy(df_total)

print("Número de dados totais: ", len(df_total_t))
print("Número de dados filtrados: ", len(df_total_t.dropna()))
print("Porcentagem de dados faltantes: {:.2f}".format(100*(1 - (len(df_total_t.dropna()) / len(df_total_t)))))

print()

df_total_t = copy.deepcopy(df_train_total)
print("Número de dados totais (treino): ", len(df_total_t))
print("Número de dados filtrados (treino): ", len(df_total_t.dropna()))
print("Porcentagem de dados faltantes (treino): {:.2f}".format(100*(1 - (len(df_total_t.dropna()) / len(df_total_t)))))

print()

df_total_t = copy.deepcopy(df_test_total)
print("Número de dados totais (teste): ", len(df_total_t))
print("Número de dados filtrados (teste): ", len(df_total_t.dropna()))
print("Porcentagem de dados faltantes (teste): {:.2f}".format(100*(1 - (len(df_total_t.dropna()) / len(df_total_t)))))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/tmp/ipykernel_32585/3843809049.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_innatura['qst_innatura'] = (df_innatura == 1.0).sum(axis=1)
/tmp/ipykernel_32585/3843809049.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_upf['qst_upf'] = (df_upf == 1.0).sum(axis=1)


Número de dados totais:  64664
Número de dados filtrados:  49843
Porcentagem de dados faltantes: 22.92

Número de dados totais (treino):  45264
Número de dados filtrados (treino):  34932
Porcentagem de dados faltantes (treino): 22.83

Número de dados totais (teste):  19400
Número de dados filtrados (teste):  14911
Porcentagem de dados faltantes (teste): 23.14


In [10]:
from scipy.stats import chi2_contingency
from IPython.display import display_markdown


def gen_chi2_statistics(df_i, columns_dict, key_dep_d='depression'):
    selected_g = copy.deepcopy(df_i)
    data = []

    for c in columns_dict:

        # display(Markdown("### {}".format(c['name'])))

        display_markdown('''### {}'''.format(c['name']), raw=True)

        selected = copy.deepcopy(selected_g)

        key_dep = key_dep_d
        if 'dep' in c:
            key_dep = c['dep']

        selected = selected.dropna(subset=[c['key'], key_dep])
        contingency_table = pd.crosstab(selected[c['key']], selected[key_dep])
        chi2, p_value, dof, _ = chi2_contingency(contingency_table, correction=False)

        # Para printar
        contingency_table = pd.crosstab(selected[c['key']], selected[key_dep], normalize=False)
        contingency_table_n = pd.crosstab(selected[c['key']], selected[key_dep], normalize='index')
        contingency_table_c = pd.crosstab(selected[c['key']], selected[key_dep], normalize='columns')

        display(contingency_table)
        print("Normalizado pelas linhas: ")
        display(contingency_table_n * 100)

        print()
        print("Normalizado pelas colunas: ")
        display(contingency_table_c * 100)

        print("chi2: {}, p_value: {}, dof: {}".format(chi2, p_value, dof))


        print("-------------------------------------------------------------")
        print()

        data.append([c['key'], key_dep, c['name'], chi2, p_value, dof])

    return pd.DataFrame(data, columns=['variavel1', 'variavel2', 'titulo', 'chi2', 'p_value', 'dof'])


selected_columns = [ {'key': c, 'name': c} for c in df_total.columns]
df_chi = gen_chi2_statistics(df_total, selected_columns)
df_chi.to_excel(PATH_GEN_DATA / 'chi2_variables.xlsx')

### exerc_fisico

depression,0.0,1.0
exerc_fisico,,
0.0,32263,4451
1.0,25553,2397


Normalizado pelas linhas: 


depression,0.0,1.0
exerc_fisico,,
0.0,87.876559,12.123441
1.0,91.423971,8.576029



Normalizado pelas colunas: 


...

Normalizado pelas linhas: 


depression,0.0,1.0
perc_saude,,
1.0,96.713102,3.286898
2.0,93.623988,6.376012
3.0,78.083499,21.916501
4.0,48.198433,51.801567
5.0,34.320988,65.679012



Normalizado pelas colunas: 


depression,0.0,1.0
perc_saude,,
1.0,18.372077,5.271612
2.0,62.807873,36.112734
3.0,16.983188,40.245327
4.0,1.596444,14.485981
5.0,0.240418,3.884346


chi2: 7782.8561661966405, p_value: 0.0, dof: 4
-------------------------------------------------------------



In [7]:
df_chi

,variavel1,variavel2,titulo,chi2,p_value,dof
0,exerc_fisico,depression,exerc_fisico,210.905330,8.716665e-48,1
1,tabagismo,depression,tabagismo,132.591516,1.110669e-30,1
2,cons_alcool,depression,cons_alcool,71.711059,2.491323e-17,1
3,cancer,depression,cancer,137.937857,7.519344e-32,1
4,hipertensao,depression,hipertensao,639.024747,5.445572e-141,1
5,diabetes,depression,diabetes,276.661533,4.009787e-62,1
6,cardiovascular,depression,cardiovascular,621.453083,3.611749e-137,1
7,hipercolesterolemia,depression,hipercolesterolemia,535.932766,1.446295e-118,1
8,avc,depression,avc,332.787048,2.375594e-74,1
9,artrite,depression,artrite,1017.190964,3.293074e-223,1
